In [ ]:
! pip install -qU langchain-openai langchain langchain_community langchainhub
! pip install chromadb==0.5.3
# 设置 OPENAI API KEY
# # export OPENAI_API_KEY=sk-...

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/615.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/615.7 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/615.7 kB ? eta -:--:--
   ---------------------------------------- 615.7/615.7 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [20]:
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
#from langchain_chroma import Chroma
from langchain_community.vectorstores.chroma import Chroma

In [21]:
# 按照分块做rag解析
file_path = os.path.join("data", "data.md")
with open(file_path, "r", encoding="utf-8") as file:
    docs_string = file.read()

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = text_splitter.split_text(docs_string)
print("Length: ", len(splits)) 
print(splits)



Length:  11
[Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '1. 系统监控指标'}, page_content='- **交易量**: 监控每秒交易数，确保系统承载能力。\n- **响应时间**: 监控交易的平均响应时间，确保服务性能。\n- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.1 交易失败'}, page_content='- **问题描述**: 用户发起支付后，交易未能成功完成。\n- **可能原因**:\n- 网络延迟或中断\n- 支付网关服务异常\n- **解决办法**:\n- 检查网络连接，确保支付服务的网络畅通。\n- 检查支付网关服务状态，重启服务或联系服务提供商。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.2 响应时间过长'}, page_content='- **问题描述**: 用户支付请求处理时间超过正常范围。\n- **可能原因**:\n- 系统资源不足\n- 数据库查询效率低下\n- 外部服务响应慢\n- **解决办法**:\n- 增加系统资源，如CPU、内存。\n- 优化数据库查询，使用索引，减少复杂查询。\n- 与外部服务提供商沟通，优化接口性能。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.3 系统宕机'}, page_content='- **问题描述**: 支付系统完全无法访问或服务中断。\n- **可能原因**:\n- 主机硬件故障\n- 系统软件崩溃\n- 网络设备故障\n- **解决办法**:\n- 快速切换到备用服务器。\n- 检查系统日志，定位问题原因。\n- 联系硬件供应商，进行故障排查和修复。'), Document(metadata={'Header 1': '支

In [22]:
# 将每一块做文本向量化
embedding = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key="sk-PBKfDtaoCwjgPYjt6b6fEe1c524541EaAfC267F2834103E7",openai_api_base="https://api.apiyi.com/v1")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding,persist_directory="./chroma_langchain_db")
vectorstore.persist()

C:\Users\26645\AppData\Local\Temp\ipykernel_11060\144493606.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [24]:
# 文本相似性检索
results = vectorstore.similarity_search_with_score(
    "payment 服务",
    1
)
for res, score in results:
    print(f"[相似性={score:3f}] {res.page_content} [{res.metadata}]")

[相似性=0.877175] - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890 [{'Header 1': '支付系统运维知识库', 'Header 2': '8. 业务负责人'}]


In [25]:
# 向量检索
results = vectorstore.similarity_search_by_vector_with_relevance_scores(
    embedding=embedding.embed_query("payment 服务是谁维护的？"),
    k=1,
)
for res, score in results:
    print(f"[相似性={score:3f}] {res.page_content} [{res.metadata}]")


[相似性=1.088832] - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890 [{'Header 1': '支付系统运维知识库', 'Header 2': '8. 业务负责人'}]


In [26]:
# 向量检索时会把 input Embedding 之后转成向量
from openai import OpenAI
client = OpenAI(api_key="sk-PBKfDtaoCwjgPYjt6b6fEe1c524541EaAfC267F2834103E7", base_url="https://api.apiyi.com/v1")

response = client.embeddings.create(
    input="payment 服务是谁维护的？",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.013628354296088219, -0.03324894234538078, 0.05509673431515694, 0.008762978948652744, -0.004756004083901644, -0.002200687536969781, -0.015205956064164639, 0.051093071699142456, -0.006532462779432535, -0.05854359269142151, 0.09242887794971466, 0.0045969183556735516, 0.010479780845344067, -0.04435843974351883, -0.024923449382185936, 0.007172120735049248, -0.0306240264326334, 0.038843464106321335, -0.041998665779829025, -0.017658529803156853, 0.0033822315745055676, 0.03955934941768646, 0.0053525762632489204, -0.012216467410326004, -0.04157443717122078, 0.013628354296088219, -0.048759814351797104, -0.04176003485918045, 0.022431105375289917, -0.06421765685081482, 0.06363434344530106, -0.029722539708018303, 0.05615730583667755, 0.030385397374629974, 0.03812757506966591, -0.0027160595636814833, -0.00542880455031991, -0.029669512063264847, 0.017605500295758247, -0.006834063213318586, -0.02301442064344883, -0.05578610673546791, -0.003417031606659293, -0.022775791585445404, -0.0061745196580886

In [36]:
retriever = vectorstore.as_retriever

prompt = langchain_hub.pull("rlm/rag-prompt")

def format_docs(docs):
    print("匹配到的运维知识库内容：\n", "\n\n".join(doc.page_content for doc in docs))
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key="sk-PBKfDtaoCwjgPYjt6b6fEe1c524541EaAfC267F2834103E7",openai_api_base="https://api.apiyi.com/v1")

rag_chain = (
    {"context": retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("payment 服务是谁维护的？")

print("\n\nLLM 回答：", res)

c:\Users\26645\.conda\envs\aiops\Lib\site-packages\langsmith\client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


匹配到的运维知识库内容：
 - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890

- **问题描述**: 用户发起支付后，交易未能成功完成。
- **可能原因**:
- 网络延迟或中断
- 支付网关服务异常
- **解决办法**:
- 检查网络连接，确保支付服务的网络畅通。
- 检查支付网关服务状态，重启服务或联系服务提供商。

- **问题描述**: 支付系统完全无法访问或服务中断。
- **可能原因**:
- 主机硬件故障
- 系统软件崩溃
- 网络设备故障
- **解决办法**:
- 快速切换到备用服务器。
- 检查系统日志，定位问题原因。
- 联系硬件供应商，进行故障排查和修复。

- **访问控制**: 严格控制系统访问权限，实行最小权限原则。
- **数据加密**: 对敏感数据进行加密处理，保护用户隐私。
- **安全审计**: 定期进行安全审计，检查潜在的安全风险。


LLM 回答： page_content='- **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890' metadata={'Header 1': '支付系统运维知识库', 'Header 2': '8. 业务负责人'}
